In [1]:
import os
print(os.getcwd())
os.chdir("..")
print(os.getcwd())

/Users/nikhil/Documents/GitHub_portfolio/1-Enterprise-Grade RAG System/notebook
/Users/nikhil/Documents/GitHub_portfolio/1-Enterprise-Grade RAG System


In [3]:
%%capture
!pip install -r requirements.txt

In [5]:
from unstructured.partition.pdf import partition_pdf
import re

In [7]:
def load_data(file_path):
    tables = []
    text = []

    match = re.search(r"(\d{4})", file_path)
    

    elements = partition_pdf(
        filename=file_path,
        infer_table_structure=True,
        extract_image_from_pdf=False,
        chunking_strategy="by_title",
        max_characters=4000,
        new_after_n_chars=3800,
        combine_text_under_n_chars=2000,
    )

    return elements
                         
                         
            
            

In [9]:
file_path = "./data/2015-cocacola-10k-filing.pdf"
elements = load_data(file_path)

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


In [17]:
print(elements[0].text)

UNITED STATES SECURITIES AND EXCHANGE COMMISSION

Washington, D.C. 20549

FORM 10-K

ý ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934

For the fiscal year ended December 31, 2014

OR

o TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934

For the transition period from to

Commission File No. 001-02217

The Cex Company

(Exact name of Registrant as specified in its charter)

DELAWARE

58-0628465

(State or other jurisdiction of incorporation or organization)

(IRS Employer Identification No.)

One Coca-Cola Plaza

Atlanta, Georgia

30313

(Address of principal executive offices)

(Zip Code)

Registrant's telephone number, including area code: (404) 676-2121

Securities registered pursuant to Section 12(b) of the Act:

Title of each class

Name of each exchange on which registered

COMMON STOCK, $0.25 PAR VALUE

NEW YORK STOCK EXCHANGE

Securities registered pursuant to Section 12(g) of the Act: None

___________

In [19]:
category_counts ={}
for el in elements:
    category = str(type(el))
    if category in category_counts:
        category_counts[category]+=1
    else:
        category_counts[category]=1
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 260,
 "<class 'unstructured.documents.elements.TableChunk'>": 14}

In [39]:
elements[0].metadata.to_dict()

{'filetype': 'application/pdf',
 'languages': ['eng'],
 'last_modified': '2025-05-23T15:02:54',
 'page_number': 1,
 'orig_elements': 'eJzNXOtv20a2/1cGxgJ1AI8w70e/+TrOrm8bO7Cd2+22RTAv2tzKoipSTdzF/u/3zJBUnEhuKxVSmw+BzxGHGs3vvM8hv/vPUZqmhzTr3tXx6Et0ZLQSTqSIKa0MFsIqbLk12MmgdBI0MC2OTtDRQ+pcdJ2DNf85Ck2ziPXMdakt9NQ9Nsvu3X2q7+474DBGCKwZ2O/r2N0Dl+rCnTf1rMvrvvtOGD5hJ4gyPiE/nKAVbcXEZJpSYyd8E6OsAM5R+9h26SH/kjf1hzS9mbuQjv4LH8TUpdDVzexdmLq2fTdfNB4uIxOpqDZwQVVPU/c4T2Xtm9dHZcOzu6W7K7/qu6M0uzv6oXDb7t1DE+uqTuXMGGESE4kZv6XyS8K+lOWI5rDy3Wz54NMi/9q8iS59yOdx9Pby4vb8Jbq5Pb09v0E352dvry9uL+DP08uX6PyfZ/84vfz7OTq7ev364ubm4uoy327c223dTctP+hw6nbzSlDIskvZYsOSxpcJgq2SKWivvBdsbdBoQKMjoic3IDDSjdsIyDUK0ie6v3w04RajWBwbuG9fe17O7rpmdoJeTswliRAr7FJ5Lt1i4rv453eYlG2ByhMSQqMOGGcDKK45tjB4TG0hFYgUKqPcHEzcTCTBkdApMAy1FD5uFD8wGuly/I0yM80PD9Orq+jWiBH/1uxSHSsqM9BVOPBAsXEzYGqqwS5WMkiuppdobImDK4HSY1hNZDrwnQS1Eb+CYnNBNjLJgR4snQMYOjMj3S0KqCPbt8u3p1+j6/M3V9S168/b65u3p5S26vcpG8BYsHaIcXV0jKo/jC3T1Ct3+4/ypfVzZxtOz2/wxtVxspX3CG2u4Stg54gBrC1

In [43]:
from pydantic import BaseModel
from typing import Any
class Element(BaseModel):
    text:str
    type : Any
    metadata :dict

In [122]:
def extract_metadata(el):
    meta = el.metadata.to_dict() if hasattr(el.metadata, "to_dict") else dict(el.metadata)

    file_name = meta.get('filename', 'unknown')
    pageno = meta.get('page_number', -1)

    # Extract year and enrich metadata
    match = re.search(r"(\d{4})", file_name)
    if match:
        meta=({
            'year': int(match.group(1)),
            'company': 'coca-cola',
            'filings': '10-K filings',
            'filename': file_name,
            'page_number': pageno
        })

    # Extract section (e.g., ITEM 1A. RISK FACTORS)
    text = el.text.replace("\n", " ").replace("  ", " ")
    match = re.search(r"(ITEM\s+\d+[A-Z]?\.\s+[A-Z\s,&\-]+)", text, re.IGNORECASE)

    meta['section'] = match.group(0).strip() if match else "unknown"

    return meta
    

In [124]:
categorized_elements =[]
for el in elements:
    if "unstructured.documents.elements.Table" in str(type(el)):
        categorized_elements.append(Element(text=str(el),type="Table",metadata=extract_metadata(el)))
    elif "unstructured.documents.elements.CompositeElement" in str(type(el)):
        categorized_elements.append(Element(text=str(el),type="text",metadata=extract_metadata(el)))

In [126]:
table_elements = [e for e in categorized_elements if e.type == "Table"]
print(len(table_elements))

# Text
text_elements = [e for e in categorized_elements if e.type == "text"]
print(len(text_elements))

14
260


In [128]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import OllamaLLM


In [130]:
prompt_text = """You are an assistant tasked with summarizing tables and text. 
Give a concise summary of the table or text. Table or text chunk: {element} """
prompt = ChatPromptTemplate.from_template(prompt_text)

# Summary chain
model = OllamaLLM(model="deepseek-r1:1.5b")
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

In [132]:
from langchain_core.documents import Document
import uuid

# Step 1: Get table texts
tables = [el.text for el in table_elements]

# Step 2: Get metadata and generate UUIDs
metadatas = [
    el.metadata.to_dict() if hasattr(el.metadata, "to_dict") else dict(el.metadata)
    for el in table_elements
]
table_ids = [str(uuid.uuid4()) for _ in tables]

# Step 3: Add doc_ids to metadata
metadatas = [
    {**metadata, "doc_id": doc_id}
    for metadata, doc_id in zip(metadatas, table_ids)
]

# Step 4: Summarize
table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

# Step 5: Create LangChain Document objects
summary_tables = [
    Document(page_content=summary, metadata={**metadata, "summary": True})
    for summary, metadata in zip(table_summaries, metadatas)
]


In [133]:


# Step 1: Prepare data
texts = [el.text for el in text_elements]
metadatas = [
    el.metadata.to_dict() if hasattr(el.metadata, "to_dict") else dict(el.metadata)
    for el in text_elements
]
doc_ids = [str(uuid.uuid4()) for _ in texts]

# Step 2: Add doc_ids to metadata
metadatas = [
    {**metadata, "doc_id": doc_id}
    for metadata, doc_id in zip(metadatas, doc_ids)
]

# Step 3: Summarize
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})

# Step 4: Create summary documents
summary_documents = [
    Document(page_content=summary, metadata=metadata)
    for summary, metadata in zip(text_summaries, metadatas)
]


In [134]:
from langchain.storage import InMemoryStore
store = InMemoryStore()
id_key = "doc_id"
store.mset(list(zip(table_ids, tables)))
store.mset(list(zip(doc_ids, texts)))

In [135]:
summary_tables

[Document(metadata={'year': 2015, 'company': 'coca-cola', 'filings': '10-K filings', 'filename': '2015-cocacola-10k-filing.pdf', 'page_number': 146, 'section': 'unknown', 'doc_id': 'ed924b48-e4ed-49c0-bcf6-ffdbca2bbb06', 'summary': True}, page_content="<think>\nAlright, so I need to summarize these exhibits related to Coca-Cola. Let me see... The user provided a list of exhibits under certain file numbers and specified that they're under File No. 001-02217. Each exhibit is numbered from 3.1 to 48, each with references to specific sections or forms.\n\nFirst, I should go through each exhibit one by one to understand what each one includes. They all seem to be related to Coca-Cola's financial reports, indentures, and notes. The main ones are the Current Report on Form 10-Q (for Q3 2012), a Registration Statement S-3 (from Oct 25, 1993), several Indenture forms, and some Form of Note entries.\n\nI notice that each exhibit is numbered and has specific dates or form references. For example,

In [ ]:
def prepare_documents(docs, is_summary=True):
    prepared_docs = []
    for doc in docs:
        metadata = dict(doc.metadata)  # copy
        metadata["doc_id"] = metadata.get("doc_id", str(uuid.uuid4()))
        metadata["summary"] = is_summary
        prepared_docs.append(Document(page_content=doc.page_content, metadata=metadata))
    return prepared_docs

In [145]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
hf = HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5')
vectorstore = Chroma(collection_name="summaries", embedding_function=hf)




In [147]:


vectorstore.add_documents(summary_documents)

['57ec63d7-b8ef-4afc-88ad-fd0ddadca863',
 'ddf8ad43-b9b9-466b-89cf-36b18bb1149c',
 '3292dec9-62e4-4ccc-84aa-6d7873d52ceb',
 'dbf3d883-24cd-4990-aae0-af024d6a18cb',
 '92680c0f-6582-42a4-8acd-d1c4b7205aa3',
 'f71b45c0-89f3-4a27-a229-163e2e15ed55',
 '0c207253-1ce1-46dd-a8b5-caddb5dd6cd7',
 '3d8adbe4-26fa-4c78-86da-667f7d2e7ac8',
 '5fb53585-0813-43f2-8afd-7d22ac2f4dc9',
 'c7a6d839-c4b7-4a93-8ab1-01def93093d6',
 '41834142-444f-40eb-bf27-4f865d505280',
 '9c8580d3-5801-4fb5-91c4-73c0942d1ff4',
 '6356f716-ddf1-4dd6-91ff-caaa42f85692',
 '9c2d62f0-23c7-40ad-bd75-99f1a1512627',
 '5b69efd0-c425-4f0d-b4a6-48704d1240d0',
 '2ef5a774-9d45-492e-9004-2d41626077b4',
 '24e6dbc7-dba4-4ce2-a9cf-9527bb15a694',
 'c24a8fbd-3517-4430-842a-74e559450da1',
 '6b3c8e35-d287-4fc5-975b-dbeb597ba197',
 '59d23a3a-500a-4d6f-a52f-c59dd67c75f6',
 '0e0f5f5a-e029-43af-86cc-be5a0e105f65',
 'aa81bd88-7a4c-4db6-bc83-92f06ba6ddf6',
 '5e179712-66de-43ed-8c31-61feb2d8a6e2',
 'ecb03eb4-b45b-47b5-be91-efbe33ed4f46',
 'ce02fd4b-fa33-

In [149]:
vectorstore.add_documents(summary_tables)

['345d9a7c-2b05-4e82-bde8-1f705c473899',
 '1dd62003-08b4-4be9-9b68-e387b1d4c01e',
 '2159106e-dab6-40ee-9e75-e487f48ef09c',
 'ff671812-8d6f-4565-aa06-da5b7cfc3425',
 'c044e509-693c-4d1b-8022-a8d42d21d21d',
 '4c522ede-63bf-4fe2-b63e-ce5f306370fe',
 'd5fb95ec-e1de-4479-b5b7-4c4eac50f668',
 'f5e082db-dfec-43d3-8178-fb2a3d24e885',
 'ce7975c8-ffa9-4a85-859f-11f74ff95e5e',
 '0951b7ed-e538-43d1-b65d-871636e73faf',
 '091befbd-5f52-43c9-b436-3f412f1e8f5b',
 '02599010-2655-48a6-af9b-28cf404282eb',
 '05793481-9b10-4f41-91d4-9f149c32805d',
 '0221a289-5237-4258-95af-01fda1b67272']

In [151]:
filtered_retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5,
        "filter": {
            "year": 2015
        }
    }
)

In [207]:

from langchain.retrievers.multi_vector import MultiVectorRetriever

retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

def ans(query):
    match = re.search(r'(\d{4})',query)
    year = int(match.group(0))
    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=store,
        id_key=id_key,
        k=4,
        search_kwargs = {'filter':{'year':year+1
                                    }})
    
    ans =retriever.invoke(query)
    return ans
        



In [209]:
query="What was Coca-Cola’s diluted net income per share for the full year 2014?"
y=ans(query)

In [211]:
for a in y:
    print(a)
    print('-'*100)

Quarterly Data (Unaudited)

First Second Third Fourth Quarter Quarter Quarter Quarter $ 10,576 $ 12,574 $ 11,976 $ 10,872 $ 6,493 7,755 7,346 6,515 1,619 2,595 2,114 770 $ 0.37 $ 0.59 $ 0.48 $ 0.18 $ $ 0.36 $ 0.58 $ 0.48 $ 0.17 $ $ 11,035 $ 12,749 $ 12,030 $ 11,040 $ 6,711 7,760 7,237 6,725 1,751 2,676 2,447 1,710 $ 0.39 $ 0.60 $ 0.55 $ 0.39 $ $ 0.39 $ 0.59 $ 0.54 $ 0.38 $ Full Year 45,998 28,109 1.62 1.60 46,854 28,433 1.94 1.90

(In millions except per share data)

2014

Net operating revenues

Gross profit

Net income attributable to shareowners of

The Coca-Cola Company

Basic net income per share

Diluted net income per share

2013

Net operating revenues

Gross profit

Net income attributable to shareowners of

The Coca-Cola Company

Basic net income per share

Diluted net income per share

1 The sum of the quarterly net income per share amounts do not agree to the full year net income per share amounts. We calculate net income per share based on the weighted-average number of ou